# Viewing the Data

In [ ]:
%matplotlib inline

import glob
import matplotlib.pyplot as plt
plt.ioff()

import blib
blib.useTheme("dark")

import src.radarkit as radarkit
import src.radarkit.chart as chart

pp = chart.ChartRHI()

In [ ]:
files = sorted(glob.glob("data/moment/20231004/*.xz"))

assert len(files) > 0, "No files found"

file = files[0]

print(f"Selected file {file}")

In [ ]:
sweep = radarkit.sweep.Sweep(file)
pp.set_data(sweep)

In [ ]:
pp.fig